<a href="https://colab.research.google.com/github/i1idan/schizophrenia-diagnosis-eeg-signals/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Clone the Repository

In [ ]:
!git clone https://github.com/i1idan/schizophrenia-diagnosis-eeg-signals.git

In [ ]:
import os
os.chdir('/content/schizophrenia-diagnosis-eeg-signals')
!git pull origin main

# Install Requirements

In [ ]:
!pip install -r requirements.txt

# Get Data

In [ ]:
!gdown --id 1jnWHWrArzJQIvny0cQkfPP42hEJAp_56
!mv DATA.mat /content/schizophrenia-diagnosis-eeg-signals/data/DATA.mat

# Training Models

In [ ]:
# Number of training in a sequence
multi_train = 10
epochs = 200
model_name = "FFTCustom"
data_path = "./data/DATA.mat"
checkpoints = "/content/drive/MyDrive/schizophrenia/checkpoints"
# checkpoints = "./checkpoints"
batch_size = 4
early_stopping = 100
reduce_lr = 50
seed = 1234

In [ ]:
csv_files = []
for n in range(multi_train):
    dir_name = f"{n}"
    new_seed = seed + n
    # preserve reproducibility
    !PYTHONHASHSEED=0
    !TF_DETERMINISTIC_OPS=0
    !TF_CUDNN_DETERMINISTIC=0
    !python train.py --model-name $model_name --epochs $epochs --seed $new_seed --dir-name $dir_name --checkpoints $checkpoints --batch-size $batch_size
    print(f"-----------------------------train {n} is done! ----------------------------------")

# Get Metrics

In [ ]:
from utils.group_metrics import get_mean_std
import os

csv_files = [os.path.join(checkpoints, model_name ,f"{n}", "log.csv") for n in range(multi_train)]
metrics = get_mean_std(csv_files, 
                       arguments=("accuracy", "loss", "val_accuracy", "val_loss"),
                       operators=(max, min, max, min)
                      )
print(metrics)